In [1]:
import pandas as pd
import numpy as np
import requests

In [11]:
r = requests.get('http://www.epfl.ch')

In [20]:
r.headers['content-type']

'text/html; charset=UTF-8'

In [20]:
df = pd.read_csv('/dlabdata1/lugeon/websites_alexa_mostpop_finalemb.gz', index_col=0).iloc[:10]

In [21]:
df

,url,cat0,embeddings
0,http://xvideos.com,Adult,"[-0.05504573384920756, 0.059303520868221916, 0..."
1,http://xnxx.com,Adult,"[0.10081154232223828, -0.054761250813802086, 0..."


In [7]:
df.url.values

array(['http://xvideos.com', 'http://xnxx.com', 'http://chaturbate.com',
       'http://youporn.com', 'http://livejasmin.com', 'http://4chan.org',
       'http://bongacams.com', 'http://nhentai.net',
       'http://adultfriendfinder.com', 'http://fetlife.com'], dtype=object)

In [11]:
a = np.ones(3)
b = np.zeros(2)

In [13]:
np.concatenate((a, b), axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [19]:
np.array([]).shape[0]

0